In [1]:
from mdptoolbox import mdp

In [2]:
import numpy as np

In [3]:
height = 2
width = 2
actions = ["left", "right", "up", "down", "drop", "pick"]
actions_count = len(actions)
items = ["blue", "red", "white"]
items_count = len(items)
# frequency of the occurency of items.
percentages = [0.5, 0.3, 0.2]
#  status of warehouse should be ["empty", "blue", "red", "white"]
tasks = ["store", "restore"]
tasks_count = len(tasks)
actions_store = ["pick", "in", "drop", "out"]
actions_restore = ["in", "pick", "out", "drop"]
warehouse_states = (items_count+1)**(height*width)
states_total = warehouse_states * tasks_count * items_count
robot_count = height * width
final_states = states_total * robot_count
# the external place, if there is no item to be stored, it equals 0, 
# otherwise it equals to the positive frequency of occurency of the item if store, and negative if restore
# the load position can only be reached from place (0,0) and the cost from there can be ignored
# store = 0
# the frequency of occurency of the item the robot carrys, 0 means robot has nothing in the hand
# load = 0 

In [4]:
def put_robot_into_state(robot_position_x, robot_position_y, state):
    r = robot_position_x*width + robot_position_y
    return r*states_total+state

In [5]:
def get_robot_and_state(my_state):
    n = my_state // states_total
    state = my_state % states_total
    x = n // width
    y = n % width
    return (x, y, state)

In [6]:
def get_state_info(s):
    n = s // warehouse_states
    task = n // items_count
    item = n % items_count
    state = s - (items_count * task + item) * warehouse_states
    return (task,item,state)

In [7]:
def convert_info_to_state(warehouse, task, item):
    state = convertWarehouseToState(warehouse)
    times = (items_count*task)+item
    return warehouse_states*times+state

In [8]:
# define the function to translate current warehouse into state index 
def convertWarehouseToState(warehouse):
    my_list = []
    for i in range(height):
        my_list.extend(warehouse[i])
    my_list.reverse()
    state = 0
    for i in range(len(my_list)-1,-1,-1):
        state = state + my_list[i]*((items_count+1) ** (i))
    return state

In [9]:
# define the function to translate current state index into warehouse
def convertStateToWarehouse(state):
    my_list = [] 
    while True:
        s = state // (items_count + 1)
        rest = state % (items_count + 1)
        my_list.append(rest)
        if s == 0:
            break
        state = s
    while len(my_list) < width*height:
        my_list.append(0)
    my_list.reverse()
    warehouse = []
    for i in range(height):
        tmp = []
        for j in range(width):
            tmp.append(my_list[i*width+j])
        warehouse.append(tmp)
    return warehouse

In [10]:
def check_moveable(action, x, y):
    if actions[action] == "left" and x == 0:
        return False
    if actions[action] == "right" and x == width - 1:
        return False
    if actions[action] == "up" and y == height - 1:
        return False
    if actions[action] == "down" and y == 0:
        return False
    return True

In [11]:
def take_move(action, x, y):
    if actions[action] == "left":
        if x == 0:
            return (x, y)
        else:
            return (x-1, y)
    if actions[action] == "right":
        if x == width - 1:
            return (x, y)
        else:
            return (x+1, y)
    if actions[action] == "up":
        if y == height - 1:
            return (x, y)
        else:
            return (x, y+1)
    if actions[action] == "down":
        if y == 0:
            return (x, y)
        else:
            return (x, y-1)
    return (x,y)

In [50]:
def check_state_validity(default_index, state, task, item, warehouse, robot_position_x, robot_position_y):
    punish = -100
    max_path = height + width - 2
    flag = [[0.0 for i in range(final_states)]for j in range(actions_count)]
    factor = [0 for i in range(actions_count)]
    r = [0 for i in range(actions_count)]
    # iterate the actions "left","right","up","down"
    for i in range(4):
        # for every state, it is in generall allowed to move one step in all four directions, without change the status of the warehouse
        # for illegal movement, the state remains.
        (x,y) = take_move(i, robot_position_x, robot_position_y)
        index = put_robot_into_state(x, y, state)
        if flag[i][index] != 0:
            factor[i] = factor[i] - flag[i][index]
        flag[i][index] = 1
        factor[i] = factor[i] + flag[i][index]
        if check_moveable(i, robot_position_x, robot_position_y):
            r[i] = -1
        else:
            r[i] = punish
    if tasks[task] == "store":
        full = height*width
        # looking for an empty place to drop
        for x in range(width):
            for y in range(height):
                if warehouse[x][y] != 0:
                    full = full - 1
        a = actions.index("drop")
        if warehouse[robot_position_x][robot_position_y] == 0:
            warehouse[robot_position_x][robot_position_y] = item+1
            for j in range(tasks_count):
                if full == 1 and tasks[j] == "store":
                    continue
                # warehouse had only one place left, after "drop" it cannot accept another "store" task
                # if not, there is no limit to get new task    
                for k in range(items_count):
                    i = convert_info_to_state(warehouse, j, k)
                    index = put_robot_into_state(0, 0, i)
                    if flag[a][index] != 0:
                        factor[a] = factor[a] - flag[a][index]
                    flag[a][index] = percentages[k]
                    factor[a] = factor[a] + flag[a][index]
            # reward depends on the positon of the robot
            r[a] = max_path - robot_position_x - robot_position_y
            # reset the warehouse
            warehouse[robot_position_x][robot_position_y] = 0
        else:
            if flag[a][default_index] != 0:
                factor[a] = factor[a] - flag[a][default_index]
            flag[a][default_index] = 1
            factor[a] = factor[a] + flag[a][default_index]
            r[a] = punish

                    
        #for x in range(height):
        #    for y in range(width):
                # status in warehouse : ["empty", "blue", "red", "white"], warehouse[x][y] = 0 means empty, 1 means blue. 
                # Therefore warehouse[x][y] == item+1 interprete the status that this place contains items[item]
        #        if warehouse[x][y] != 0:
                    #warehouse[x][y] = item+1
                    # r[a] = (max_path - x - y) * percentages[item]
                    # after "drop", it is allowed to get new task under current warehouse status, e.g. from eeeesr to reeesb via drop
                    # For eeeesr--drop-->reeesb, the robot is expected to do the "go out and pick new item and come back" by itself. This process is ignored in this program
                    #for j in range(tasks_count):
                    #    for k in range(items_count):
                    #        i = convert_info_to_state(warehouse, j, k)
                    #        index = put_robot_into_state(0, 0, i)
                    #        flag[a][index] = percentages[k]
                    #        factor[a] = factor[a] + flag[a][index]
                    # reset the warehouse status, controls that only one place will be changed
                    #warehouse[x][y] = 0
                    # no "pick" action is allowed in the warehouse, while "store"
                # if current place is not empty, keep moving - "drop" is not allowed at this place
                #else:
        #            full = full - 1
        # if full == 1: # there was only one place left
            #for x in range(height):
            #    for y in range(width):
            #        if warehouse[x][y] == 0:
        #    warehouse[robot_position_x][robot_position_y] = item+1
                        # r[actions.index("drop")] = (max_path - x - y) * percentages[item]
                        # after the warehouse is full, there is no more "store" task allowed
       #     for j in range(items_count):
       #         i = convert_info_to_state(warehouse, task, j)
       #         index = put_robot_into_state(0, 0, i)
       #         for a in range(actions_count):
       #             factor[a] = factor[a] - flag[a][index]
       #             flag[a][index] = 0
       #     warehouse[x][y] = 0
        if full == 0: # there was no place left
            a = actions.index("drop")
            if flag[a][default_index] != 0:
                factor[a] = factor[a] - flag[a][default_index]
            flag[a][default_index] = 1
            factor[a] = factor[a] + flag[a][default_index]
            r[a] = punish
        # for task "store", action "pick" will not change the state
        a = actions.index("pick")
        if flag[a][default_index] != 0:
            factor[a] = factor[a] - flag[a][default_index]
        flag[a][default_index] = 1
        factor[a] = factor[a] + flag[a][default_index]
        r[a] = punish
        # every other states remain unreachable.
        # e.g. from eeeesr, there is no way to reach rrresr
    
    
    
    elif tasks[task] == "restore":
        # looking for a place contains the same item
        found = [0 for i in range(items_count)]
        empty = 1
        for x in range(width):
            for y in range(height):
                if warehouse[x][y] != 0:
                    found[warehouse[x][y]-1] = found[warehouse[x][y]-1]+1
        if warehouse[robot_position_x][robot_position_y] == item+1:
            empty = 0
            found[item] = found[item]-1
            warehouse[robot_position_x][robot_position_y] = 0
            a = actions.index("pick")
            for k in range(items_count):
                        # since there was item removed, there will be no problem to get a new "store" task
                i = convert_info_to_state(warehouse, tasks.index("store"), k)
                index = put_robot_into_state(0, 0, i)
                if flag[a][index] != 0:
                    factor[a] = factor[a] - flag[a][index]
                flag[a][index] = percentages[k]
                factor[a] = factor[a] + flag[a][index]
                        # for "restore", it is only allowed if required item can be found in the warehouse
                        # e.g. switch from eebrrb to eeerrr trough "drop" is allowed, but to eeerrb is not allowed
                if found[k] != 0:
                    i = convert_info_to_state(warehouse, task, k)
                    index = put_robot_into_state(0, 0, i)
                    if flag[a][index] != 0:
                        factor[a] = factor[a] - flag[a][index]
                    flag[a][index] = percentages[k]
                    factor[a] = factor[a] + flag[a][index]
                    # reset the warehouse status, controls that only one place will be changed
            r[a] = max_path - robot_position_x - robot_position_y
            warehouse[x][y] = item+1
            
            
        #for x in range(height):
        #    for y in range(width):
                # status in warehouse : ["empty", "blue", "red", "white"], warehouse[x][y] = 0 means empty, 1 means blue. 
                # Therefore warehouse[x][y] == item+1 interprete the status that this place contains items[item]
        #        if warehouse[x][y] != 0:
        #            found[warehouse[x][y]-1] = found[warehouse[x][y]-1]+1
        #        if warehouse[x][y] == item+1:
        #            empty = 0
        #            found[item] = found[item]-1
        #            warehouse[x][y] = 0
                    # index = convert_info_to_state(warehouse, task, item)
        #            a = actions.index("pick")
        #            r[a] = max_path - robot_position_x - robot_position_y
                    # after "pick", it is allowed to get new task under current warehouse status, e.g. from reeerr to eeeesb via drop
                    # For reeerr--pick-->eeeesb, the robot is expected to do the "go out and pick new item and come back" by itself. This process is ignored in this program
        #            for k in range(items_count):
                        # since there was item removed, there will be no problem to get a new "store" task
        #                i = convert_info_to_state(warehouse, tasks.index("store"), k)
        #                index = put_robot_into_state(0, 0, i)
        #                flag[a][index] = percentages[k]
        #                factor[a] = factor[a] + flag[a][index]
                        # for "restore", it is only allowed if required item can be found in the warehouse
                        # e.g. switch from eebrrb to eeerrr trough "drop" is allowed, but to eeerrb is not allowed
        #                if found[k] != 0:
        #                    i = convert_info_to_state(warehouse, task, k)
        #                    index = put_robot_into_state(0, 0, i)
        #                    flag[a][index] = percentages[k]
        #                    factor[a] = factor[a] + flag[a][index]
                    # reset the warehouse status, controls that only one place will be changed
        #            warehouse[x][y] = item+1
                    # no "drop" action is allowed in the warehouse, while "restore"
                # if current place does not contains the required item, keep moving - "pick" is not allowed at this place
        if empty == 1: # the warehouse didnot contain required item
            a = actions.index("pick")
            if flag[a][index] != 0:
                factor[a] = factor[a] - flag[a][index]
            flag[a][default_index] = 1
            factor[a] = factor[a] + flag[a][default_index]
            r[a] = punish
        # for task "restore", action "drop" will not change the state
        a = actions.index("drop")
        if flag[a][default_index] != 0:
            factor[a] = factor[a] - flag[a][default_index]
        flag[a][default_index] = 1
        factor[a] = factor[a] + flag[a][default_index]
        r[a] = punish
        # every other states remain unreachable.
        # e.g. from eeeerr, there is no way to reach rrrerr
    return (flag, factor, r)

In [51]:
def update_policy(states_validity, P, s, factor):
    for u in range(len(states_validity)):
        if factor[u] == 0:
            print("???????????????????", "action:", u,"state:", s)
            return P
        factor_inverse = 1/factor[u]
        count = 0
        prob_count = 0
        for v in range(len(states_validity[0])):
            if states_validity[u][v] != 0:
                count = count + states_validity[u][v]
                prob = states_validity[u][v] * factor_inverse
                #prob = round(prob, 2)
                #if states_validity[u][v] * factor < prob:
                #    prob = prob - 0.01
                P[u][s][v] = prob
                prob_count = prob_count + prob
        print("factor:", factor, "count:", count, "prob_count:", prob_count)
    return P

In [52]:
def getTransitionAndRewardArrays():
    P = np.array([[[0.0 for s1 in range(final_states)]for s in range(final_states)] for a in range(actions_count)])
    R = np.array([[0 for a in range(actions_count)]for a in range(final_states)])
    #R = []
    max_steps = ((width-1)*width/2)*height+((height-1)*height/2)*width
    # iterrate the state_total. e.g. 0 is empty warehouse with task store and item blue, 4**4 is empty warehouse with task store and item red
    for s in range(final_states):
        (robot_position_x, robot_position_y, st) = get_robot_and_state(s)
        (task, item, state) = get_state_info(st)
        warehouse = convertStateToWarehouse(state)
        # find all states that are reachable through a certain action
        # get a 2 dimentional flag, for [u][v] = 1, means it is possible to change state s into v through the action u
        #for i in range(width):
        #    for j in range(height):
        (states_validity, factor, reward) = check_state_validity(s, st, task, item, warehouse, robot_position_x, robot_position_y)
        P = update_policy(states_validity, P, s, factor)
        R[s] = reward
                #R = get_reward(s, task, item, warehouse, R, i, j)
        #R.append(r)
        # if there is a way, the policy value equals to the frequency of the occurency of the item
            
            
        #print("########################################################")
        # update the policy matrix according to the result of validity check
        #P = update_policy(states_validity, P, s, factor)
    return(P, R)

In [53]:
(P, R) = getTransitionAndRewardArrays()

factor: [1, 1, 1, 1, 2.0, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 2.0, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 2.0, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 2.0, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 2.0, 1] count: 2.0 prob_count: 1.0
factor: [1, 1, 1, 1, 2.0, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 2.0, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 2.0, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 2.0, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 2.0, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 2.0, 1] count: 2.0 prob_count: 1.0
factor: [1, 1, 1, 1, 2.0, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 2.0, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 2.0, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 2.0, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 2.0, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 2.0, 1] count: 2.0 prob_count: 1.0
factor: [1, 1, 1, 1, 2.0, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 2

factor: [1, 1, 1, 1, 1.0, 1] count: 1.0 prob_count: 1.0
factor: [1, 1, 1, 1, 1.0, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 2.0, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 2.0, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 2.0, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 2.0, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 2.0, 1] count: 2.0 prob_count: 1.0
factor: [1, 1, 1, 1, 2.0, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1.0, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1.0, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1.0, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1.0, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1.0, 1] count: 1.0 prob_count: 1.0
factor: [1, 1, 1, 1, 1.0, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1.0, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1.0, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1.0, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1.0, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1

factor: [1, 1, 1, 1, 1.0, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1.0, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1.0, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1.0, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1.0, 1] count: 1.0 prob_count: 1.0
factor: [1, 1, 1, 1, 1.0, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1.0, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1.0, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1.0, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1.0, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1.0, 1] count: 1.0 prob_count: 1.0
factor: [1, 1, 1, 1, 1.0, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 2.0, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 2.0, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 2.0, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 2.0, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 2.0, 1] count: 2.0 prob_count: 1.0
factor: [1, 1, 1, 1, 2.0, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1

factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 

factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 

...

factor: [1, 1, 1, 1, 1, 1.8] count: 1.8 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: 

factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1.5] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1.5] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1.5] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1.5] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1.5] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1.5] count: 1.5 prob_count: 0.9999999999999999
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1

factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1.5] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1.5] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1.5] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1.5] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1.5] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1.5] count: 1.5 prob_count: 0.9999999999999999
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1

factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1.2] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1.2] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1.2] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1.2] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1.2] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1.2] count: 1.2 prob_count: 1.0000000000000002
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1

factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1.5] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1.5] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1.5] count: 1 prob_count: 1.0
factor: [1, 1, 1, 1, 1, 1.5] count: 1 prob_count: 1.0
fact

In [54]:
print(R)

[[-100   -1   -1 -100    2 -100]
 [-100   -1   -1 -100    2 -100]
 [-100   -1   -1 -100    2 -100]
 ...
 [  -1 -100 -100   -1 -100 -100]
 [  -1 -100 -100   -1 -100 -100]
 [  -1 -100 -100   -1 -100    0]]


In [55]:
print(P)

[[[1. 0. 0. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 0. 0.]
  [0. 0. 1. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 1. 0. 0.]
  [0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 0. 0. 1.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 1. 0. 0.]
  [0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 0. 0. 1.]]

 [[1. 0. 0. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 0. 0.]
  [0. 0. 1. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 1. 0. 0.]
  [0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 0. 0. 1.]]

 [[1. 0. 0. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 0. 0.]
  [0. 0. 1. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 1. 0. 0.]
  [0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]


In [56]:
def isStochastic(matrix):
    """Check that ``matrix`` is row stochastic.
    Returns
    =======
    is_stochastic : bool
        ``True`` if ``matrix`` is row stochastic, ``False`` otherwise.
    """
    try:
        absdiff = (np.abs(matrix.sum(axis=1) - np.ones(matrix.shape[0])))
        print(len(np.abs(matrix.sum(axis=1))))
        print(np.abs(matrix.sum(axis=1)))
        print(np.ones(matrix.shape[0]))
        print(absdiff)
    except AttributeError:
        matrix = np.array(matrix)
        absdiff = (np.abs(matrix.sum(axis=1) - np.ones(matrix.shape[0])))
    print(np.spacing(np.float64(1)))
    print(absdiff.max())
    return (absdiff.max() <= 10*np.spacing(np.float64(1)))

In [57]:
for i in range(actions_count):
    print("action",i,":", isStochastic(P[i]))

6144
[1. 1. 1. ... 1. 1. 1.]
[1. 1. 1. ... 1. 1. 1.]
[0. 0. 0. ... 0. 0. 0.]
2.220446049250313e-16
0.0
action 0 : True
6144
[1. 1. 1. ... 1. 1. 1.]
[1. 1. 1. ... 1. 1. 1.]
[0. 0. 0. ... 0. 0. 0.]
2.220446049250313e-16
0.0
action 1 : True
6144
[1. 1. 1. ... 1. 1. 1.]
[1. 1. 1. ... 1. 1. 1.]
[0. 0. 0. ... 0. 0. 0.]
2.220446049250313e-16
0.0
action 2 : True
6144
[1. 1. 1. ... 1. 1. 1.]
[1. 1. 1. ... 1. 1. 1.]
[0. 0. 0. ... 0. 0. 0.]
2.220446049250313e-16
0.0
action 3 : True
6144
[1. 1. 1. ... 1. 1. 1.]
[1. 1. 1. ... 1. 1. 1.]
[0. 0. 0. ... 0. 0. 0.]
2.220446049250313e-16
0.0
action 4 : True
6144
[1. 1. 1. ... 1. 1. 1.]
[1. 1. 1. ... 1. 1. 1.]
[0. 0. 0. ... 0. 0. 0.]
2.220446049250313e-16
2.220446049250313e-16
action 5 : True


In [58]:
# Definition of the mdp with discount factor, maximal iterations, the tranisition probability matrix and the reward matrix
mdpresultPolicy = mdp.PolicyIteration(P,R,0.3, max_iter=100)
mdpresultValue = mdp.ValueIteration(P,R,0.3, max_iter=100)

# Run the MDP
mdpresultPolicy.run()
mdpresultValue.run()

In [59]:
# print the result
print('PolicyIteration:')
print(mdpresultPolicy.policy)
print(mdpresultPolicy.V)
print(mdpresultPolicy.iter)

print('ValueIteration:')
print(mdpresultValue.policy)
print(mdpresultValue.V)
print(mdpresultValue.iter)

PolicyIteration:
(4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 1, 2, 1, 2, 2, 2, 2

In [60]:
def interpret_policy(policy, index):
    (x, y, s) = get_robot_and_state(index)
    (task, item, state) = get_state_info(s)
    warehouse = convertStateToWarehouse(state)
    print("current position:", x, y)
    print("current warehouse:", warehouse)
    print("current task", tasks[task])
    print("required item:", items[item])
    print("chosen action:", actions[policy])

In [62]:
for i in range(0, final_states, 1000):
    print("-----------PolicyIteration:---------------")
    interpret_policy(mdpresultPolicy.policy[i], i)
    print("-----------ValueIteration:---------------")
    interpret_policy(mdpresultValue.policy[i], i)

-----------PolicyIteration:---------------
current position: 0 0
current warehouse: [[0, 0], [0, 0]]
current task store
required item: blue
chosen action: drop
-----------ValueIteration:---------------
current position: 0 0
current warehouse: [[0, 0], [0, 0]]
current task store
required item: blue
chosen action: drop
-----------PolicyIteration:---------------
current position: 0 0
current warehouse: [[3, 2], [2, 0]]
current task restore
required item: blue
chosen action: right
-----------ValueIteration:---------------
current position: 0 0
current warehouse: [[3, 2], [2, 0]]
current task restore
required item: blue
chosen action: right
-----------PolicyIteration:---------------
current position: 0 1
current warehouse: [[3, 1], [0, 0]]
current task store
required item: red
chosen action: right
-----------ValueIteration:---------------
current position: 0 1
current warehouse: [[3, 1], [0, 0]]
current task store
required item: red
chosen action: right
-----------PolicyIteration:----------

In [64]:
for i in range(final_states):
    if mdpresultPolicy.policy[i] != mdpresultValue.policy[i]:
        print("-----------PolicyIteration:---------------")
        interpret_policy(mdpresultPolicy.policy[i], i)
        print("-----------ValueIteration:---------------")
        interpret_policy(mdpresultValue.policy[i], i)

-----------PolicyIteration:---------------
current position: 0 0
current warehouse: [[1, 0], [0, 2]]
current task store
required item: red
chosen action: up
-----------ValueIteration:---------------
current position: 0 0
current warehouse: [[1, 0], [0, 2]]
current task store
required item: red
chosen action: right
-----------PolicyIteration:---------------
current position: 0 0
current warehouse: [[2, 0], [0, 1]]
current task store
required item: red
chosen action: up
-----------ValueIteration:---------------
current position: 0 0
current warehouse: [[2, 0], [0, 1]]
current task store
required item: red
chosen action: right
-----------PolicyIteration:---------------
current position: 0 0
current warehouse: [[2, 0], [0, 2]]
current task store
required item: red
chosen action: up
-----------ValueIteration:---------------
current position: 0 0
current warehouse: [[2, 0], [0, 2]]
current task store
required item: red
chosen action: right
-----------PolicyIteration:---------------
current p

current task restore
required item: white
chosen action: left
-----------PolicyIteration:---------------
current position: 1 1
current warehouse: [[3, 1], [2, 1]]
current task restore
required item: white
chosen action: down
-----------ValueIteration:---------------
current position: 1 1
current warehouse: [[3, 1], [2, 1]]
current task restore
required item: white
chosen action: left
-----------PolicyIteration:---------------
current position: 1 1
current warehouse: [[3, 3], [3, 1]]
current task restore
required item: white
chosen action: down
-----------ValueIteration:---------------
current position: 1 1
current warehouse: [[3, 3], [3, 1]]
current task restore
required item: white
chosen action: left
-----------PolicyIteration:---------------
current position: 1 1
current warehouse: [[3, 3], [3, 2]]
current task restore
required item: white
chosen action: down
-----------ValueIteration:---------------
current position: 1 1
current warehouse: [[3, 3], [3, 2]]
current task restore
requ

In [25]:
# Definition of the mdp with discount factor, maximal iterations, the tranisition probability matrix and the reward matrix
mdpresultPolicy = mdp.PolicyIteration(P,R,0.9)
mdpresultValue = mdp.ValueIteration(P,R,0.9)

# Run the MDP
mdpresultPolicy.run()
mdpresultValue.run()

In [26]:
# print the result
print('PolicyIteration:')
print(mdpresultPolicy.policy)
print(mdpresultPolicy.V)
print(mdpresultPolicy.iter)

print('ValueIteration:')
print(mdpresultValue.policy)
print(mdpresultValue.V)
print(mdpresultValue.iter)

PolicyIteration:
(4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 4, 4, 4, 4, 4, 0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 4, 4, 0, 0, 0, 0, 4, 0, 0, 0, 4, 0, 0, 0, 4, 4, 4, 4, 4, 0, 0, 0, 4, 0, 0, 0, 4, 0, 0, 0, 4, 4, 4, 4, 4, 0, 0, 0, 4, 0, 0, 0, 4, 0, 0, 0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 0, 0, 4, 0, 0, 0, 4, 0, 0, 0, 4, 4, 4, 4, 4, 0, 0, 0, 4, 0, 0, 0, 4, 0, 0, 0, 4, 4, 4, 4, 4, 0, 0, 0, 4, 0, 0, 0, 4, 0, 0, 0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 0, 0, 4, 0, 0, 0, 4, 0, 0, 0, 4, 4, 4, 4, 4, 0, 0, 0, 4, 0, 0, 0, 4, 0, 0, 0, 4, 4, 4, 4, 4, 0, 0, 0, 4, 0, 0, 0, 4, 0, 0, 0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 4, 4, 4, 4, 4, 0, 4, 0, 0, 4, 4, 0, 0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 4, 4, 4, 4, 0, 4, 4, 4, 4, 4, 4, 4, 4

In [168]:
def isStochastic(matrix):
    """Check that ``matrix`` is row stochastic.
    Returns
    =======
    is_stochastic : bool
        ``True`` if ``matrix`` is row stochastic, ``False`` otherwise.
    """
    try:
        absdiff = (np.abs(matrix.sum(axis=1) - np.ones(matrix.shape[0])))
        print(np.abs(matrix.sum(axis=1)))
        print(np.ones(matrix.shape[0]))
        print(absdiff)
    except AttributeError:
        matrix = np.array(matrix)
        absdiff = (np.abs(matrix.sum(axis=1) - np.ones(matrix.shape[0])))
    print(np.spacing(np.float64(1)))
    print(absdiff.max())
    return (absdiff.max() <= 10*np.spacing(np.float64(1)))

In [15]:
def get_reward(state, task, item, warehouse, R, robot_position_x, robot_position_y):
    # for a 2x2 warehouse, with coordinates (x,y) starts from (0,0), the shortest step is always x+y
    punish = -100
    reward = 100
    # every step get reward -1
    for a in range(4):
        R[state][a] = -1    
    # check the warehouse status
    # calculate the moving steps to reach current state
    steps = max_step
    found = [0 for i in range(items_count)]
    for x in range(height):
        for y in range(width):
            if warehouse[x][y] != 0:
                found[warehouse[x][y]-1] = found[warehouse[x][y]-1] + 1
                steps = steps - x - y
    remain_steps = max_step - steps
    count = 0
    for i in range(items_count):
        count = count + found[i]
        
    # if an item will be put dowm while store, high reward
    if tasks[task] == "store":
        a = actions.index("drop")
        # the less the remain_steps, the better for next state
        # this benefit decreases with the frequency of occurence
        pos = items_count - sorted(percentages).index(percentages[item])
        R[state][a] = (reward - remain_steps) * pos
        a = actions.index("pick")
        R[state][a] = punish
    # if an item will be chosen while restore, high reward
    elif tasks[task] == "restore":
        a = actions.index("pick")
        # the less it took to reach current state, the better the next state can be
        # this benefit decreases with the frequency of occurence
        pos = items_count - sorted(percentages).index(percentages[item])
        R[state][a] = (reward - steps) * pos
        a = actions.index("drop")
        R[state][a] = punish
    # getting restore task while the warehouse is empty, does not make sence, therefore punishment for every action
    if count == 0 and task == tasks.index("restore"):
        for a in range(actions_count):
            R[state][a] = punish
    # getting store task while the warehouse is full, does not make sence, therefore punishment for every action
    if count == height * width and task == tasks.index("store"):
        for a in range(actions_count):
            R[state][a] = punish
    # getting restore task while the warehouse does not contains required item, does not make sence, therefore punishment for every action
    if found[item] == 0 and task == tasks.index("restore"):
        for a in range(actions_count):
            R[state][a] = punish
    return R